In [4]:
# ============================================================
# Mental Health Support Chatbot (DistilGPT-2 Fine-Tuned)
# ============================================================

!pip install -q transformers datasets accelerate torch pyarrow

import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import random

# ---------------------------
# Configuration
# ---------------------------
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./mental_health_bot"
MAX_LENGTH = 256
TRAIN_SAMPLES = 5000   # Reduced for faster training
BATCH_SIZE = 4

# ---------------------------
# Create a Mental Health Dataset
# ---------------------------
print("Creating mental health conversation dataset...")

mental_health_conversations = [
    # Anxiety-related conversations
    {"context": "I've been feeling really anxious lately", "response": "I understand anxiety can be really difficult. Have you tried any relaxation techniques like deep breathing?"},
    {"context": "My anxiety is overwhelming me", "response": "When anxiety feels overwhelming, try focusing on your breathing. Inhale for 4 counts, hold for 4, exhale for 6."},
    {"context": "I'm having panic attacks", "response": "Panic attacks are scary but temporary. Try grounding techniques like naming 5 things you can see, 4 you can touch, 3 you can hear, 2 you can smell, and 1 you can taste."},

    # Depression-related conversations
    {"context": "I feel depressed most days", "response": "Depression is really hard. Remember that it's okay to not be okay. Have you considered speaking with a professional?"},
    {"context": "I don't enjoy anything anymore", "response": "Loss of interest can be a sign of depression. Start small - try to do one small thing you used to enjoy, even for just 5 minutes."},
    {"context": "Everything feels hopeless", "response": "When things feel hopeless, it's important to remember that feelings are temporary. You've gotten through hard times before."},

    # Sleep issues
    {"context": "I can't sleep at night", "response": "Sleep issues are common with stress. Try establishing a consistent bedtime routine and avoid screens an hour before bed."},
    {"context": "I wake up with anxiety", "response": "Morning anxiety can be tough. Try writing down your thoughts or doing gentle stretching when you wake up."},

    # Loneliness
    {"context": "I feel lonely and isolated", "response": "Loneliness is a painful feeling. Would you like to talk about what's going on? Sometimes just sharing helps."},
    {"context": "I feel like no one understands me", "response": "Feeling misunderstood is painful. I'm here to listen without judgment."},

    # Stress
    {"context": "Work has been really stressful", "response": "Work stress can be overwhelming. Remember to take regular breaks and practice self-care."},
    {"context": "I'm overwhelmed with responsibilities", "response": "When overwhelmed, try breaking tasks into smaller steps and prioritizing what's most important."},

    # Motivation
    {"context": "I'm struggling with motivation", "response": "Lack of motivation can be tough. Be gentle with yourself. Even small accomplishments are worth celebrating."},
    {"context": "I can't seem to get anything done", "response": "Sometimes just getting started is the hardest part. Try setting a timer for 5 minutes and see what you can accomplish."},

    # Self-care
    {"context": "I don't take care of myself", "response": "Self-care is important. Start with basic needs - are you drinking enough water and getting some movement?"},
    {"context": "I'm too hard on myself", "response": "Try speaking to yourself as you would to a friend. You deserve the same kindness you give others."},

    # General support
    {"context": "I don't know what to do", "response": "It's okay to not have all the answers. Let's take things one step at a time."},
    {"context": "I feel stuck", "response": "When you feel stuck, sometimes changing your environment can help. Try going for a short walk or doing something different."},
    {"context": "I'm scared", "response": "Fear is a natural emotion. What's one small step you can take to feel a bit safer?"},
    {"context": "I need help", "response": "Asking for help is a sign of strength. I'm here to support you."}
]

# Duplicate and augment the dataset
print("Augmenting dataset...")
augmented_data = []
for conversation in mental_health_conversations:
    # Add original
    augmented_data.append(conversation)

    # Add variations
    context = conversation["context"]
    response = conversation["response"]

    # Add slight variations
    variations = [
        {"context": context.lower(), "response": response},
        {"context": context.upper(), "response": response},
        {"context": context + ".", "response": response},
        {"context": "I'm feeling like " + context[2:] if context.startswith("I ") else context, "response": response},
    ]
    augmented_data.extend(variations)

# Create more samples by combining
for i in range(len(mental_health_conversations) * 10):
    context_item = random.choice(mental_health_conversations)
    response_item = random.choice(mental_health_conversations)
    if random.random() > 0.3:  # 70% chance to use matching pairs
        augmented_data.append({
            "context": context_item["context"],
            "response": response_item["response"]
        })

print(f"Created dataset with {len(augmented_data)} conversation pairs")

# Create Hugging Face Dataset
dataset = Dataset.from_list(augmented_data)

# Split into train (80%) and validation (20%)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# ---------------------------
# Load Model & Tokenizer
# ---------------------------
print("\nLoading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# ---------------------------
# Preprocessing Function
# ---------------------------
def preprocess_function(examples):
    # Create conversation pairs
    texts = []
    for context, response in zip(examples['context'], examples['response']):
        # Format: User: [context]\nBot: [response]\n
        text = f"User: {context}\nBot: {response}\n"
        texts.append(text)

    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

    # Set labels for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# ---------------------------
# Prepare Dataset
# ---------------------------
print("\nPreprocessing dataset...")
train_tokenized = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

val_tokenized = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print(f"Training samples after preprocessing: {len(train_tokenized)}")
print(f"Validation samples after preprocessing: {len(val_tokenized)}")

# ---------------------------
# Training Arguments
# ---------------------------
print("\nSetting up training...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=3e-5,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    save_total_limit=2,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    warmup_steps=50
)

# ---------------------------
# Data Collator
# ---------------------------
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# ---------------------------
# Trainer
# ---------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

# ---------------------------
# Train
# ---------------------------
print("\nStarting training...")
trainer.train()

# ---------------------------
# Save Model
# ---------------------------
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"\n✅ Model saved to {OUTPUT_DIR}")

# ---------------------------
# Reload Model for Inference
# ---------------------------
print("\nLoading model for inference...")
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

# ---------------------------
# Response Generator
# ---------------------------
def generate_response(user_input, max_new_tokens=100):
    prompt = f"User: {user_input}\nBot:"

    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_LENGTH
    ).to(device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.8,
            top_p=0.92,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
            repetition_penalty=1.1,
            num_return_sequences=1
        )

    # Decode response
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the bot's response
    if "Bot:" in full_text:
        response = full_text.split("Bot:")[-1].strip()
    else:
        response = full_text.replace(prompt, "").strip()

    # Clean up response
    if "\nUser:" in response:
        response = response.split("\nUser:")[0].strip()

    return response

# ---------------------------
# Test the Model
# ---------------------------
print("\n" + "="*60)
print("🤖 Testing the trained Mental Health Support Bot...")
print("="*60)

test_inputs = [
    "I've been feeling really anxious",
    "I can't sleep at night",
    "I feel so alone",
    "Everything feels hopeless",
    "I need help"
]

print("\nSample conversations:")
print("-" * 50)
for test_input in test_inputs:
    response = generate_response(test_input)
    print(f"👤 You: {test_input}")
    print(f"🤖 Bot: {response}")
    print("-" * 50)

# ---------------------------
# Enhanced Chatbot Interface
# ---------------------------
def chat_interface():
    print("\n" + "="*60)
    print("🧠 MENTAL HEALTH SUPPORT CHATBOT")
    print("="*60)
    print("Commands:")
    print("  'exit' - End conversation")
    print("  'help' - Show conversation tips")
    print("  'clear' - Clear conversation history")
    print("  'suggest' - Get conversation starters")
    print("="*60)

    conversation_history = []
    conversation_starters = [
        "I've been feeling really anxious",
        "I can't stop worrying",
        "I feel lonely",
        "I'm having trouble sleeping",
        "I don't enjoy anything anymore",
        "I feel overwhelmed",
        "I'm stressed about work/school",
        "I feel like crying",
        "I don't want to get out of bed",
        "I'm scared about the future"
    ]

    print("\n💬 Hi, I'm here to listen and support you. How are you feeling today?")

    while True:
        try:
            print("\n" + "─" * 40)
            user_input = input("👤 You: ").strip()

            if not user_input:
                print("🤖 Bot: I'm still here. Take your time.")
                continue

            # Handle commands
            if user_input.lower() in ['exit', 'quit', 'bye', 'goodbye']:
                print("\n🤖 Bot: Remember to be kind to yourself. You're doing the best you can. ❤️")
                print("      If you need immediate help, contact:")
                print("      National Suicide Prevention Lifeline: 988 (US)")
                print("      Crisis Text Line: Text HOME to 741741")
                break

            elif user_input.lower() == 'help':
                print("\n🤖 Bot: I'm here to listen and support you. You can talk about:")
                print("      • Your feelings and emotions")
                print("      • Daily struggles and challenges")
                print("      • Anxiety, stress, or depression")
                print("      • Relationships and loneliness")
                print("      • Self-care and coping strategies")
                print("      • Anything else on your mind")
                continue

            elif user_input.lower() == 'clear':
                conversation_history = []
                print("\n🤖 Bot: Conversation cleared. I'm ready to listen.")
                continue

            elif user_input.lower() == 'suggest':
                print("\n🤖 Bot: Here are some conversation starters:")
                for i, starter in enumerate(conversation_starters, 1):
                    print(f"      {i}. {starter}")
                continue

            # Store conversation for context (optional future enhancement)
            conversation_history.append(f"User: {user_input}")

            # Generate response with typing indicator simulation
            print("🤖 Bot: ", end="", flush=True)

            response = generate_response(user_input)

            # If response is too short, add fallback
            if len(response) < 10:
                fallbacks = [
                    "I hear you. That sounds really difficult.",
                    "Thank you for sharing that with me.",
                    "I'm here with you through this.",
                    "That must be really hard. Would you like to talk more about it?",
                    "I understand. Your feelings are valid."
                ]
                response = random.choice(fallbacks)

            print(response)

            conversation_history.append(f"Bot: {response}")

        except KeyboardInterrupt:
            print("\n\n🤖 Bot: Take care of yourself. Reach out if you need to talk again. 💙")
            break
        except Exception as e:
            print(f"\n🤖 Bot: I encountered an issue. Let me try to respond differently.")
            fallback = random.choice([
                "I'm here to listen. Can you tell me more about how you're feeling?",
                "That sounds important. Would you like to elaborate?",
                "Thank you for sharing. How has that been affecting you?",
                "I understand this is difficult. What's been on your mind lately?"
            ])
            print(f"🤖 Bot: {fallback}")

# Run the chat interface
chat_interface()

# ---------------------------
# Save the complete model
# ---------------------------
print("\n" + "="*60)
print("📁 Model Information:")
print("="*60)
print(f"Model saved at: {OUTPUT_DIR}")
print(f"Model type: {MODEL_NAME}")
print(f"Training samples used: {len(train_dataset)}")
print(f"Model is ready for inference!")

# Optional: Test more examples
print("\n🎯 Additional test examples:")
print("-" * 40)
more_tests = [
    "I'm having a bad day",
    "I feel like giving up",
    "Nothing makes me happy",
    "I'm worried about everything"
]

for test in more_tests:
    response = generate_response(test)
    print(f"👤: {test}")
    print(f"🤖: {response}")
    print()

Creating mental health conversation dataset...
Augmenting dataset...
Created dataset with 236 conversation pairs
Training samples: 188
Validation samples: 48

Loading model and tokenizer...

Preprocessing dataset...


Map:   0%|          | 0/188 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Training samples after preprocessing: 188
Validation samples after preprocessing: 48

Setting up training...

Starting training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,2.200200,1.820118
100,0.678700,0.552106
150,0.449500,0.362651
200,0.345500,0.321650


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].



✅ Model saved to ./mental_health_bot

Loading model for inference...
Model loaded on cuda

🤖 Testing the trained Mental Health Support Bot...

Sample conversations:
--------------------------------------------------
👤 You: I've been feeling really anxious
🤖 Bot: Asking for help is a sign of strength. Even small accomplishments are worth celebrating.
Mentifier: Try speaking to yourself as you would to an adult and fellow robot. You deserve the same kindness, care, and support that comes with sharing.
--------------------------------------------------
👤 You: I can't sleep at night
🤖 Bot: Sleep issues are common with stress. Try establishing a consistent bedtime routine and avoid screens an hour before bed.

Bot 5a: Morning anxiety is really hard. Remember that it's okay to not have all the answers - everything you need is going well. Even small accomplishments are worth celebrating!
To help support this cause, click here . Start typing without ever knowing what your next task would be l